In [2]:
import pandas as pd
import numpy as np
import csv
import random


import keras
from keras.models import Model,Sequential
from keras import backend as K
from keras.layers.core import Flatten, Dense, Lambda
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils
from keras.layers import Convolution2D, ELU
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json

import json
import os.path

In [17]:
Driving_Data = pd.read_csv('F:/Udacity_SD/driving_log.csv', index_col = False)
Driving_Data.columns = ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']

In [19]:
def ImageCropping(image):
    
    # Function to remove the car hood on the bottom of the image and the sky part on the top of the image which are useless 
    # regressing the steering angle
    imsize = image.shape()
    
    imagetop = int(imsize[0]/5)
    imagebottom = imsize[0] - 25
    
    image = image[imagetop:imagebottom, 0:imsize[1]]
    
    #Resize to fit data to 200*66 for NVIDIA model 
    
    col,row = 200,66
    image = cv2.resize(image,(col,row), interpolation=cv2.INTER_AREA) 
    
    return image

def ImagePreProcessing():
    
    
    


In [23]:
random.random()

0.24217105675523287

In [8]:
# Defining the Neural network

def nvidia_model():
    learning_rate = 0.0001
    col,row = 200,66
    
    # Model structure following the NVIDIA paper
    
    model = Sequential()
    
    #Normalization
    model.add(Lambda(lambda x:x/127.5 -1., input_shape = (row,col,3)))
    
    #Convolution with 24 filters and 5x5 stride and a Exponential linear unit activation
    model.add(Convolution2D(24, 5, 5,border_mode='valid',subsample=(2,2),     init='he_normal'))
    model.add(ELU())
    
    #Convolution with 36 filters and 5x5 stride and a Exponential linear unit activation
    model.add(Convolution2D(36, 5, 5, border_mode='valid',subsample=(2,2), init='he_normal'))
    model.add(ELU())
    
    #Convolution with 48 filters and 5x5 stride and a Exponential linear unit activation
    model.add(Convolution2D(48, 5, 5, border_mode='valid',subsample=(2,2),init='he_normal'))
    model.add(ELU())
    
    #Convolution with 64 filters and 3x3 stride and a Exponential linear unit activation
    model.add(Convolution2D(64, 3, 3, border_mode='valid',subsample=(1,1),init='he_normal'))
    model.add(ELU())
    
    #Convolution with 64 filters and 3x3 stride and a Exponential linear unit activation
    model.add(Convolution2D(64, 3, 3, border_mode='valid',subsample=(1,1),init='he_normal'))
    model.add(ELU())
    
    #Flattening the image to get them working for the fully connected layer to regress steering angle
    model.add(Flatten())
    
    #Fully Connected layer to map all the pixels in an image
    model.add(Dense(1164,init='he_normal'))
    model.add(ELU())
    model.add(Dense(100,init='he_normal'))
    model.add(ELU())
    model.add(Dense(50,init='he_normal'))
    model.add(ELU())
    model.add(Dense(10,init='he_normal'))
    model.add(ELU())
    model.add(Dense(1,name='output',init='he_normal'))
    
    optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

    return model

In [9]:
model = nvidia_model()

W0725 22:34:37.787250 14292 deprecation_wrapper.py:119] From C:\Users\jaswa\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

C:\Users\jaswa\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), padding="valid", kernel_initializer="he_normal")`
  from ipykernel import kernelapp as app
W0725 22:34:37.833736 14292 deprecation_wrapper.py:119] From C:\Users\jaswa\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

C:\Users\jaswa\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), strides=(2, 2), padding="valid", kernel_initializer="he_normal")`
C:\Users\jaswa\Anaconda3\lib\site-packages\ipykernel_launcher.p